In [1]:
import sympy as sym
import math as m
import numpy as np
import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import SIC_POVM_Jun27 as sic


Solution to the system: [ 5.39947499e-02 -5.39947499e-02  3.41479678e-01 -3.41479678e-01
 -3.49382845e-01  3.49382845e-01  1.36024980e-01  5.43783422e-02
 -5.43783422e-02  3.37911719e-01 -3.37911719e-01 -3.59205510e-01
  3.59205510e-01  1.45687225e-01  7.02075490e-01 -7.02075490e-01
  2.27088003e-01 -2.27088003e-01 -4.98689480e-01  4.98689480e-01
  6.68160881e-01 -3.48936782e-02  3.48936782e-02 -4.66517243e-01
  4.66517243e-01  1.45544506e-02 -1.45544506e-02 -1.17615790e-01
  2.65383007e-13  1.00000000e+00 -8.95312658e-13  1.00000000e+00
  3.72621111e-13  1.00000000e+00  2.02656113e-11]
Residuals: [1.4329289114494227e-14, 8.89843754237063e-14, -2.3941606956446403e-13, -1.4241885448740277e-11, -3.5830365414834474e-12, -5.655476087440547e-13, -3.3992242188665557e-12, 1.443108797144431e-14, 9.413303470040546e-14, -2.312395045494266e-13, -1.4496293054833131e-11, -3.953847428263887e-12, -6.227240945122503e-13, -3.41774148409909e-12, 1.8631890489990362e-13, -4.3787196091216174e-13, -9.359509